## 1- Importing Libraries & Modules

In [4]:
import numpy as np
import cv2

# local module
import video
from video import presets

## 2- Defining a class for the application

In [5]:
# This class will manage the video capture and object tracking
class App(object):
    
    # Constructor function to initialize video capture
    def __init__(self, video_src):
        self.cam = video.create_capture(video_src)
        ret, self.frame = self.cam.read()
        cv2.namedWindow('camshift')

        # Initializing variables for the selection and tracking state
        self.selection = None
        self.drag_start = None
        self.tracking_state = 0
        self.show_backproj = False

## 3- Mouse Handler

In [6]:
# It allows the user to click and drag to select a region of interest (ROI) for tracking
def onmouse(self, event, x, y, flags, param):
    x, y = np.int16([x, y])  # Converting coordinates to integer type
    if event == cv2.EVENT_LBUTTONDOWN:
        self.drag_start = (x, y)
        self.tracking_state = 0

    if self.drag_start: 
        if flags & cv2.EVENT_FLAG_LBUTTON:
            h, w = self.frame.shape[:2]
            xo, yo = self.drag_start
            x0, y0 = np.maximum(0, np.minimum([xo, yo], [x, y]))
            x1, y1 = np.minimum([w, h], np.maximum([xo, yo], [x, y]))
            self.selection = None

            # Check if the selected region is valid (non-zero width and height)
            if x1-x0 > 0 and y1-y0 > 0:
                self.selection = (x0, y0, x1, y1)
            else:
                self.drag_start = None

        # If a valid region is selected, set the tracking state to 1
            if self.selection is not None:
                self.tracking_state = 1

## 4- Display Histogram of The ROI

In [7]:
def show_hist(self):
        bin_count = self.hist.shape[0]
        bin_w = 24
        img = np.zeros((256, bin_count*bin_w, 3), np.uint8)

        # Drawing rectangles to represent the histogram of the ROI
        for i in range(bin_count):
            h = int(self.hist[i])
            cv2.rectangle(img, (i*bin_w, 256), ((i+1)*bin_w, 256-h), (int(180.0*i/bin_count), 255, 255), -1)
        img = cv2.cvtColor(img, cv2.COLOR_HSV2BGR)
        cv2.imshow('hist', img)

## 5- Main loop of the application
- where the object tracking takes place

In [ ]:
def run(self):
        while True:
            ret, self.frame = self.cam.read()
            if ret == False:
                break

            vis = self.frame.copy()

            # If the user is dragging to select an area, draw a rectangle around the selection
            if self.selection:
                x0, y0, x1, y1 = self.selection
                cv2.rectangle(vis, (x0, y0), (x1, y1), (0, 255, 0), 2)

            # Handling the tracking process
            if self.tracking_state == 1:
                # Initialize tracking window and compute histogram of the selected region
                hsv = cv2.cvtColor(self.frame, cv2.COLOR_BGR2HSV)
                mask = cv2.inRange(hsv, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
                self.hist = cv2.calcHist([hsv], [0], mask, [16], [0, 180])
                cv2.normalize(self.hist, self.hist, 0, 255, cv2.NORM_MINMAX)

                # Set the tracking state to 2, indicating tracking is ongoing
                self.tracking_state = 2

            elif self.tracking_state == 2:
                # Perform back-projection to update the tracking window
                hsv = cv2.cvtColor(self.frame, cv2.COLOR_BGR2HSV)
                mask = cv2.inRange(hsv, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
                prob = cv2.calcBackProject([hsv], [0], self.hist, [0, 180], 1)
                prob &= mask
                term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

                # Apply the CamShift algorithm to find the new position of the object
                track_box, self.track_window = cv2.CamShift(prob, self.track_window, term_crit)

                # Draw the tracking result on the video
                if self.show_backproj:
                    vis[:] = prob[..., np.newaxis]
                try:
                    cv2.ellipse(vis, track_box, (0, 0, 255), 2)
                except:
                    pass

            # Show the result in the video window
            cv2.imshow('camshift', vis)

            ch = cv2.waitKey(5)
            if ch == 27:
                break
            
# Starting the application and setting up the mouse callback function
if __name__ == '__main__':
    import sys
    try: video_src = sys.argv[1]
    except: video_src = 0
    print(__doc__)
    App(video_src).run()